# M2177.003100 Deep Learning <br> Final Project: Text-to-Image Synthesis 

Copyright (C) Data Science & AI Laboratory, Seoul National University. This material is for educational uses only. Some contents are based on the material provided by other paper/book authors and may be copyrighted by them. 

**For understanding of this work, please carefully look at given PDF file.**

**Note**: certain details are missing or ambiguous on purpose, in order to test your knowledge on the related materials. However, if you really feel that something essential is missing and cannot proceed to the next step, then contact the teaching staff with clear description of your problem.

## 1. Load datasets

In [1]:
import os
os.chdir('/home/jaehyun/Downloads/22Final/')
import torch
from torch.autograd import Variable
import torchvision.transforms as transforms
from collections import OrderedDict

import numpy as np

from models import GENERATOR
from config import cfg, cfg_from_file
import pprint
import datetime
import dateutil.tz
import time
from PIL import Image

from encoders import CNN_ENCODER, RNN_ENCODER
from dataset import CUBDataset
from utils import *
from loss import *
from transformers import BertModel

2022-12-23 19:41:21.562683: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-23 19:41:21.645366: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-23 19:41:21.962677: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.3/lib64
2022-12-23 19:41:21.962724: W tensorflow/compiler/xla/stream_executor/

In [2]:
# Set a config file as 'train_birds.yml' in training, as 'eval_birds.yml' for evaluation
cfg_from_file('cfg/eval_birds.yml') # eval_birds.yml

print('Using config:')
pprint.pprint(cfg)

os.environ['CUDA_VISIBLE_DEVICES'] = cfg.GPU_ID

now = datetime.datetime.now(dateutil.tz.tzlocal())
timestamp = now.strftime('%Y_%m_%d_%H_%M_%S')
output_dir = 'sample/%s_%s_%s' % (cfg.DATASET_NAME, cfg.CONFIG_NAME, timestamp)

os.makedirs(output_dir, exist_ok=True)

Using config:
{'BATCH_SIZE': 10,
 'CA': True,
 'CHECKPOINT_DIR': './checkpoints',
 'CHECKPOINT_NAME': '',
 'CNN': {'EMBEDDING_DIM': 32, 'H_DIM': 256},
 'CONFIG_NAME': 'text-to-image',
 'CUDA': True,
 'DATASET_NAME': 'birds',
 'DATA_DIR': 'data/birds',
 'EMBEDDING_TYPE': 'cnn-rnn',
 'GAN': {'B_ATTENTION': False,
         'B_CONDITION': False,
         'B_DCGAN': False,
         'CONDITION_DIM': 100,
         'DF_DIM': 32,
         'EMBEDDING_DIM': 256,
         'GF_DIM': 32,
         'R_NUM': 2,
         'Z_DIM': 100},
 'GPU_ID': '0, 1',
 'IMAGE_SIZE': 128,
 'NUM_BATCH_FOR_TEST': 4,
 'RANDOM_SEED': 1234,
 'RNN': {'EMBEDDING_DIM': 0,
         'H_DIM': 0,
         'TYPE': 'LSTM',
         'VOCAB_SIZE': 0,
         'WORD_EMBEDDING_DIM': 0},
 'R_PRECISION_DIR': '',
 'R_PRECISION_FILE': '',
 'R_PRECISION_FILE_HIDDEN': '',
 'TEST': {'B_EXAMPLE': False,
          'GENERATED_HIDDEN_TEST_IMAGES': '',
          'GENERATED_TEST_IMAGES': './evaluation/generated_images'},
 'TEXT': {'CAPTIONS_PER_IMA

In [3]:
imsize = cfg.TREE.BASE_SIZE * (4 ** (cfg.TREE.BRANCH_NUM - 1))
image_transform = transforms.Compose([
    transforms.Resize(int(imsize)),
    transforms.RandomCrop(imsize),
    transforms.RandomHorizontalFlip()])

#### Load the dataset
test_dataset = CUBDataset(cfg.DATA_DIR, transform=image_transform, split='test')

print(f'test data directory:\n{test_dataset.split_dir}\n')
print(f'# of test filenames:{test_dataset.filenames.shape}\n')
print(f'example of filename of valid image:{test_dataset.filenames[0]}\n')
print(f'example of caption and its ids:\n{test_dataset.captions[0]}\n{test_dataset.captions_ids[0]}\n')
print(f'# of test captions:{np.asarray(test_dataset.captions).shape}\n')
print(f'# of test caption ids:{np.asarray(test_dataset.captions_ids).shape}\n')

self.current_dir:
/home/jaehyun/Downloads/22Final

self.data_dir:
/home/jaehyun/Downloads/22Final/data/birds

self.image_dir:
/home/jaehyun/Downloads/22Final/data/birds/CUB-200-2011/images

filepath /home/jaehyun/Downloads/22Final/data/birds/captions.pickle
Load from:  /home/jaehyun/Downloads/22Final/data/birds/captions.pickle
test data directory:
/home/jaehyun/Downloads/22Final/data/birds/test

# of test filenames:(2933,)

example of filename of valid image:001.Black_footed_Albatross/Black_Footed_Albatross_0046_18

example of caption and its ids:
['this', 'is', 'a', 'small', 'bird', 'that', 'has', 'a', 'brilliant', 'blue', 'color', 'on', 'it', 's', 'body', 'a', 'slightly', 'darker', 'blue', 'on', 'it', 's', 'head', 'a', 'teal', 'color', 'on', 'it', 's', 'wings', 'and', 'a', 'light', 'colored', 'beak']
[18, 19, 1, 250, 2, 33, 13, 1, 853, 50, 37, 86, 53, 54, 15, 1, 178, 31, 50, 86, 53, 54, 25, 1, 1054, 37, 86, 53, 54, 17, 8, 1, 67, 89, 10]

# of test captions:(29330,)

# of test caption

/tmp/ipykernel_2956880/3617630386.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  print(f'# of test captions:{np.asarray(test_dataset.captions).shape}\n')
/tmp/ipykernel_2956880/3617630386.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  print(f'# of test caption ids:{np.asarray(test_dataset.captions_ids).shape}\n')


In [4]:
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=cfg.BATCH_SIZE, drop_last=True, shuffle=False, num_workers=int(cfg.WORKERS),
                                              collate_fn = my_collate_fn_test)

## 2. Define models and load weights

In [5]:
# load the text encoder model to generate images for evaluation
text_encoder = RNN_ENCODER(test_dataset.n_words, nhidden=cfg.TEXT.EMBEDDING_DIM)

#######################################################
# TODO
# (1) Build Generator and Network
######################################################
netG = GENERATOR()

origin_state_dict = torch.load(os.path.join(cfg.CHECKPOINT_DIR, cfg.TRAIN.GENERATOR), map_location=lambda storage, loc: storage)

state_dict = dict()
for key in origin_state_dict.keys():
    in_state_dict = OrderedDict()
    for name, value in origin_state_dict[key].items():
        name = name.replace("module.","") # moudle이 붙은 경우 이를 제거하기 위해
        in_state_dict[name] = value
    state_dict[key] = in_state_dict


text_encoder.load_state_dict(state_dict['text_encoder'])
for p in text_encoder.parameters():
    p.requires_grad = False

netG.load_state_dict(state_dict['netG'])
for p in netG.parameters():
    p.requires_grad = False

print('Load model from:', os.path.join(cfg.CHECKPOINT_DIR, cfg.TRAIN.GENERATOR))
text_encoder.eval()
netG.eval()

#from transformers import AutoModel
#bert_encoder = AutoModel.from_pretrained("prajjwal1/bert-mini")
#bert_encoder = BertModel.from_pretrained('bert-base-uncased')
#for param in bert_encoder.parameters():
#    param.require_grad = False
#bert_encoder.eval()

bs = cfg.BATCH_SIZE
noise = Variable(torch.FloatTensor(bs, cfg.GAN.Z_DIM))

if cfg.CUDA:
    text_encoder = text_encoder.cuda()
    netG = netG.cuda()
#    bert_encoder = bert_encoder.cuda()
    noise = noise.cuda()
device = 'cuda' if torch.cuda.is_available() else 'cpu'

netG = torch.nn.DataParallel(netG)

Load model from: ./checkpoints/final_model.pth


## 3. Generate Images and save them to evaluate metrics

In [6]:
for step, data in enumerate(test_dataloader, 0):
    _, bert_enc, bert_mask, _, captions, cap_lens, class_ids, keys, sent_idx = prepare_data(data)

    #######################################################
    # TODO
    # (2) Compute Text Embeddings using RNN Encoder
    ######################################################
    hidden = text_encoder.init_hidden(captions.size(0))
    text_embedding = text_encoder(captions, cap_lens, hidden)
    #bert_embedding = bert_encoder(bert_enc)[1]
    #######################################################
    # TODO
    # (3) Generate fake images using Generator
    ######################################################
    noise.data.normal_(0, 1)

    fake_imgs, _, _ = netG(noise, text_embedding[1], z2w=False)
    fake_imgs = fake_imgs[2]

    for j in range(bs):
        if not os.path.exists(os.path.join(cfg.TEST.GENERATED_TEST_IMAGES, keys[j].split('/')[0])):
            os.makedirs(os.path.join(cfg.TEST.GENERATED_TEST_IMAGES, keys[j].split('/')[0]))

        im = fake_imgs[j].data.cpu().numpy()
        im = (im + 1.0) * 127.5
        im = im.astype(np.uint8)
        im = np.transpose(im, (1, 2, 0))
        im = Image.fromarray(im)
        print(os.path.join(cfg.TEST.GENERATED_TEST_IMAGES, keys[j] + '_{}.png'.format(sent_idx[j])))
        im.save(os.path.join(cfg.TEST.GENERATED_TEST_IMAGES, keys[j] + '_{}.png'.format(sent_idx[j])))

/home/jaehyun/anaconda3/envs/geometry/lib/python3.9/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0046_18_0.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0046_18_1.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0046_18_2.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0046_18_3.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0046_18_4.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0046_18_5.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0046_18_6.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0046_18_7.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0046_18_8.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0046_18_9.png
./evaluation/generated_images/001.Black_footed_Albatross/Bla

./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0025_796057_0.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0025_796057_1.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0025_796057_2.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0025_796057_3.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0025_796057_4.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0025_796057_5.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0025_796057_6.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0025_796057_7.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0025_796057_8.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0025_796057_9.png
./evaluation/generat

./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0042_796071_0.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0042_796071_1.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0042_796071_2.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0042_796071_3.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0042_796071_4.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0042_796071_5.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0042_796071_6.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0042_796071_7.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0042_796071_8.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0042_796071_9.png
./evaluation/generat

./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0005_796090_0.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0005_796090_1.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0005_796090_2.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0005_796090_3.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0005_796090_4.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0005_796090_5.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0005_796090_6.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0005_796090_7.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0005_796090_8.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0005_796090_9.png
./evaluation/generat

...

./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0019_188460_0.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0019_188460_1.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0019_188460_2.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0019_188460_3.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0019_188460_4.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0019_188460_5.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0019_188460_6.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0019_188460_7.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0019_188460_8.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0019_188460_9.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0103_188483_0.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0103_188483_1.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0103_188483_2.png
./evaluation/generated_images/197.Mars

./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0076_188108_0.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0076_188108_1.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0076_188108_2.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0076_188108_3.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0076_188108_4.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0076_188108_5.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0076_188108_6.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0076_188108_7.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0076_188108_8.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0076_188108_9.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0071_188111_0.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0071_188111_1.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0071_188111_2.png
./evaluation/generated_images/197.Mars

./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0042_188195_0.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0042_188195_1.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0042_188195_2.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0042_188195_3.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0042_188195_4.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0042_188195_5.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0042_188195_6.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0042_188195_7.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0042_188195_8.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0042_188195_9.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0094_188710_0.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0094_188710_1.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0094_188710_2.png
./evaluation/generated_images/197.Mars

./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0044_188270_0.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0044_188270_1.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0044_188270_2.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0044_188270_3.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0044_188270_4.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0044_188270_5.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0044_188270_6.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0044_188270_7.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0044_188270_8.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0044_188270_9.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0012_188275_0.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0012_188275_1.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0012_188275_2.png
./evaluation/generated_images/197.Mars

./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0147_188367_0.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0147_188367_1.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0147_188367_2.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0147_188367_3.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0147_188367_4.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0147_188367_5.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0147_188367_6.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0147_188367_7.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0147_188367_8.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0147_188367_9.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0048_188370_0.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0048_188370_1.png
./evaluation/generated_images/197.Marsh_Wren/Marsh_Wren_0048_188370_2.png
./evaluation/generated_images/197.Mars